In [5]:
import keras.losses
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array
from tensorflow.keras import layers
from keras.regularizers import l2
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from sklearn.utils import class_weight
import datetime

In [2]:
model = VGG16(include_top=True)

In [3]:
def get_features(ID):
    img_path = 'food_porn/food/' + ID + '.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = np.array(model.predict(x))
    return features

In [6]:
image_1 = image.load_img('food_porn/food/' + '00050' + '.jpg', target_size=(224, 224))
image_s = img_to_array(image_1)
print(image_1)
print('____________________')
print(image_s)

<PIL.Image.Image image mode=RGB size=224x224 at 0x1ED81D0E520>
____________________
[[[200. 200. 236.]
  [194. 194. 230.]
  [197. 201. 236.]
  ...
  [156. 169. 113.]
  [159. 178. 123.]
  [149. 173. 123.]]

 [[201. 201. 237.]
  [193. 193. 229.]
  [199. 203. 238.]
  ...
  [157. 175. 117.]
  [164. 185. 129.]
  [137. 164. 113.]]

 [[191. 191. 227.]
  [188. 191. 226.]
  [187. 191. 226.]
  ...
  [139. 167. 106.]
  [150. 179. 123.]
  [141. 174. 121.]]

 ...

 [[165. 191. 128.]
  [127. 156.  92.]
  [130. 159.  95.]
  ...
  [189. 222. 253.]
  [176. 205. 237.]
  [186. 203. 229.]]

 [[145. 177. 112.]
  [140. 175. 109.]
  [120. 155.  89.]
  ...
  [187. 222. 250.]
  [176. 206. 230.]
  [180. 197. 207.]]

 [[140. 177. 110.]
  [149. 186. 119.]
  [147. 184. 117.]
  ...
  [187. 223. 249.]
  [180. 209. 225.]
  [177. 193. 193.]]]


In [17]:
%%time
print(decode_predictions(get_features('00000')))


40960/35363 [==================================] - 0s 0us/step
[[('n04476259', 'tray', 0.27396187), ('n03485794', 'handkerchief', 0.23914962), ('n03291819', 'envelope', 0.18719405), ('n03871628', 'packet', 0.06075603), ('n03961711', 'plate_rack', 0.026308354)]]
Wall time: 1.62 s


In [4]:
%%time
data = np.zeros((10000,1001))
id = np.array(range(0,10000))
data[:,0] = id.transpose()

def features():
    for i in range(0,10000):
        featureid = '{0:05}'.format(i)
        features = get_features(featureid)
        data[i,1:] = features
        if i % 100 == True:
            print(i)
    return data

data = features()

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
7401
7501
7601
7701
7801
7901
8001
8101
8201
8301
8401
8501
8601
8701
8801
8901
9001
9101
9201
9301
9401
9501
9601
9701
9801
9901


In [6]:
save_ass = 'features_vgg16.csv'

In [5]:
print(np.count_nonzero(data))
save_ass = 'features_vgg16.csv'

np.savetxt(save_ass, data, delimiter=',')

10009999


using false we got:     527 correct
using true we got:      520 correct

how about... we use the true version to feed a NN

In [7]:
ABC1 = np.loadtxt("train_triplets.txt").astype('int') 

data = np.genfromtxt(save_ass, delimiter= ',')
data = np.array(data[:,1:])

def load_features(int_id):
    return data[int_id]

In [8]:
%%time

# There are 4 different permutations of ABC that we can know the answer of. (CAB and CBA we don't know)
ONE_SET_SIZE = 59515

#create the base dataset with the 2 sets of labels
data_y1 = np.ones((ONE_SET_SIZE * 2,1))
data_y0 = np.zeros((ONE_SET_SIZE * 2,1))

#create the inverse dataset and a set where B is connected to A (and inv)

ACB0 = np.zeros((ONE_SET_SIZE,3)).astype('int')    # One needs 3 different assignments since the arrays would be linked otherwise.
BAC1 = np.zeros((ONE_SET_SIZE,3)).astype('int')
BCA0 = np.zeros((ONE_SET_SIZE,3)).astype('int')

ACB0[:,0] = ABC1[:,0]
BAC1[:,1] = ABC1[:,0]
BCA0[:,2] = ABC1[:,0]

ACB0[:,2] = ABC1[:,1]
BAC1[:,0] = ABC1[:,1]
BCA0[:,0] = ABC1[:,1]

ACB0[:,1] = ABC1[:,2]
BAC1[:,2] = ABC1[:,2]
BCA0[:,1] = ABC1[:,2]

#assemble everything into one huge pile of S*** in order to shuffle it
id_fin = np.concatenate((ABC1, BAC1, ACB0, BCA0),axis=0)
labels = np.concatenate((data_y1,data_y0),axis=0)
id_labels = np.concatenate((id_fin,labels),axis=1)

data_nn_x = np.zeros((ONE_SET_SIZE * 4,3000))

for i in range(0,3):
        for j in range(0,ONE_SET_SIZE * 4):
            data_nn_x[j,range(i*1000,1000*(i+1))] = load_features(id_labels[j,i].astype('int'))
            if j%50000==False:
                print("we're at entry:", i, j)


#print(train_trip)
#print(train_trip_inv)
#now we try to randomize the data in order to increase our accuracy
# we may even double the dataset by using 0 as well as the 1 case

we're at entry: 0 0
we're at entry: 0 50000
we're at entry: 0 100000
we're at entry: 0 150000
we're at entry: 0 200000
we're at entry: 1 0
we're at entry: 1 50000
we're at entry: 1 100000
we're at entry: 1 150000
we're at entry: 1 200000
we're at entry: 2 0
we're at entry: 2 50000
we're at entry: 2 100000
we're at entry: 2 150000
we're at entry: 2 200000
Wall time: 1min 3s


In [9]:
%%time
collab = np.concatenate((data_nn_x,labels), axis=1)
np.random.shuffle(collab)

data_x = collab[:,:3000]
data_y = collab[:,3000]

Wall time: 32.9 s


In [ ]:
#print("%d bytes" % (data_nn_x.size * data_nn_x.itemsize))
#np.savetxt("nn_data.csv",data_nn_x,delimiter=',')

In [12]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
layer = layers.Dense(
    units=64,
    kernel_regularizer=regularizers.l1_l2(l1=1e-2, l2=1e-2),
    bias_regularizer=regularizers.l2(1e-2),
    activity_regularizer=regularizers.l2(1e-2)
)

In [13]:

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
%%time

#TAKE IT Branda:
neuralNetwork = Sequential()
neuralNetwork.add(Dense(1024, activation='relu', input_dim=3000, kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.5))
neuralNetwork.add(Dense(512, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.4))
neuralNetwork.add(Dense(128, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.3))
neuralNetwork.add(Dense(128, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.3))
neuralNetwork.add(Dense(64, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(1, activation='sigmoid'))
#neuralNetwork.add(BatchNormalization())

neuralNetwork.compile(loss = keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

#fit the network to (for now the un-sparse matrix)
neuralNetwork.fit(data_x, data_y, epochs=128, batch_size=1024, verbose=1, validation_split = 0.2)

#evaluation of the network prediction
#predict_test_nn = neuralNetwork.predict(df_test_onehot_nn)
#predict_train_nn = neuralNetwork.predict(df_train_onehot_nn)
#print(predict_test_nn)
#print(predict_train_nn)
#predict_test_nn = (predict_test_nn >= 0.5)
neuralNetwork.summary()

In [15]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 3360), started 3 days, 0:14:59 ago. (Use '!kill 3360' to kill it.)

In [28]:
test_trip = np.loadtxt("test_triplets.txt").astype('int')
test_nn_x = np.zeros((59544,3000))
for i in range(0,3):
        for j in range(0,59544):
            test_nn_x[j,range(i*1000,1000*(i+1))] = load_features(test_trip[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)

predict_test_nn = neuralNetwork.predict(test_nn_x)

we're at entry: 0 0
we're at entry: 0 10000
we're at entry: 0 20000
we're at entry: 0 30000
we're at entry: 0 40000
we're at entry: 0 50000
we're at entry: 1 0
we're at entry: 1 10000
we're at entry: 1 20000
we're at entry: 1 30000
we're at entry: 1 40000
we're at entry: 1 50000
we're at entry: 2 0
we're at entry: 2 10000
we're at entry: 2 20000
we're at entry: 2 30000
we're at entry: 2 40000
we're at entry: 2 50000


In [29]:
export = np.array(predict_test_nn>0.5).astype('int')
np.savetxt("submission_vgg16_regularizer.txt",np.round(export,decimals=0), fmt='%i')